**update 20250225**

polygon转point时出现时间对照不上问题，没有按照date格式直接保存。
--> geopandas转换时无法转换为date,手动在arcgis中调整
增加最低采样点数

In [1]:

import geopandas as gpd
import shapely
import random
import datetime

In [2]:
def compute_sample_count(area, min_samples=8, max_samples=100):
    """根据多边形面积计算采样点数量，确保数量不会太多"""
    scaled_samples = int(area * 300)  # 可以调整比例因子
    return max(min_samples, min(scaled_samples, max_samples))  # 限制范围

In [3]:
def generate_sample_points(geometry, num_samples):
    """在多边形内部生成随机点"""
    minx, miny, maxx, maxy = geometry.bounds
    points = []
    while len(points) < num_samples:
        p = shapely.geometry.Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if geometry.contains(p):  # 确保点在多边形内
            points.append(p)
    return points

In [4]:
# Load the shapefile with geopandas
shapefile_path = r"E:\Sea Ice Classification\AI4ArcticASIP Sea ice Dataset version2\2018.910\Sample_20180910\Sample_20180910_lead_addition.shp"
gdf = gpd.read_file(shapefile_path)

In [5]:
# 创建采样点并保留属性
sampled_points = []
for _, row in gdf.iterrows():
    num_samples = compute_sample_count(row.geometry.area)  # 计算需要生成的点数
    points = generate_sample_points(row.geometry, num_samples)
   
    for point in points:
       
        sampled_points.append({
            "geometry": point,
            "source": row["source"],
            "time_t": row["time"],
            "stage": row["stage"],
            "class": row["class"],
            "class_name": row["class_name"],
            "CC": row["CC"],
            "form": row["form"]
        })

# 转换为 GeoDataFrame
points_gdf = gpd.GeoDataFrame(sampled_points, crs=gdf.crs)

# 保存 Shapefile（适用于 GEE）
output_shp_path = "E:/Sea Ice Classification/AI4ArcticASIP Sea ice Dataset version2/2018.910/Sample_20180910/Sample_20180910_lead_addition_Points.shp"
points_gdf.to_file(output_shp_path)

# 也可以保存为 CSV（适用于 GEE）
output_csv_path = "E:/Sea Ice Classification/AI4ArcticASIP Sea ice Dataset version2/2018.910/Sample_20180910/Sample_20180910_lead_addition_Points.csv"
points_gdf.drop(columns="geometry").to_csv(output_csv_path, index=False)

print(f"采样点 Shapefile 已保存到: {output_shp_path}")
print(f"采样点 CSV 已保存到: {output_csv_path}")

采样点 Shapefile 已保存到: E:/Sea Ice Classification/AI4ArcticASIP Sea ice Dataset version2/2018.910/Sample_20180910/Sample_20180910_lead_addition_Points.shp
采样点 CSV 已保存到: E:/Sea Ice Classification/AI4ArcticASIP Sea ice Dataset version2/2018.910/Sample_20180910/Sample_20180910_lead_addition_Points.csv


In [7]:
print(points_gdf['time_t'].dtype)  # 检查 time 列的类型

object
